# Lock-in: Control y adquisición de señales

In [1]:
#Generación de gráficas y .csv 
#Datos de referencia: fase, frecuencia y amplitud
#Datos de ganacia y constante de tiempo: sensibilidad y constante de tiempo
#Datos de display y output: Elección de canal (CH1), Display: R / CH2: Theta

In [3]:
import pyvisa
import time
import numpy as np
import csv
import time

In [4]:
def adquirir(device, q):
    device.write(q)
    data=''
    try:
        while True:
            b=device.read_bytes(1)
            data=data+str(b)
    except:
       # print('Lectura terminada')
        pass
    data=data.replace('b','')
    data=data.replace('\'','')
    data=data.replace(r'\r','')
    return data

In [5]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())
lockin = rm.open_resource('ASRL1::INSTR', timeout=100)
voltimeter = rm.open_resource('ASRL5::INSTR')

('ASRL1::INSTR', 'ASRL5::INSTR')


Datos de entrada

In [10]:
n = 1
tiempo_estabilizacion=23
n_barridos = 1
lista_freq = [1, 2] # 5, 10,20, 50, 100, 200]
# Ajustar Fase de Referencia
lockin.write("PHAS {}".format(0))
time.sleep(0.2)
# Ajustar Amplitud de Referencia
lockin.write("SLVL {}".format(4.24))
time.sleep(1)
# Ajustar Constante de tiempo a 1 s
lockin.write("OELT {}".format(10))
time.sleep(1)
# Ajustar Sensibilidad a 50 mV/nA
lockin.write("SENS {}".format(22))

9

Realizar n barridos de captura de datos

In [11]:
for barrido in range(1, n_barridos+1):
    start_time = time.time()
    print('barrido ' + str(barrido))
    # Para cada frecuencia, tomar n muestras
    # para cada frecuencia en la lista de frecuencias
    tabla_muestras=[]
    for freq in lista_freq:
        print('freq ' + str(freq))
        #Ajustar frecuencia de referencia
        lockin.write("FREQ {}".format(freq))
        time.sleep(tiempo_estabilizacion)
        lista_muestras=[]
        lista_muestras.append(freq)
        # repetir n veces
        for i in range(n):
            # adquirir muestra y asignarlo en R y theta
            R = float(adquirir (lockin, 'OUTP? 3'))
            # print('R: ' + str(R))
            time.sleep(1)
            lista_muestras.append(R)
            theta = float(adquirir (lockin, 'OUTP? 4'))
            # print('Theta: ' + str(theta))
            time.sleep(1)
            lista_muestras.append(theta)
            volt = float(adquirir(voltimeter, ':SENSe:DATA:FRESh?')[5:15])
            lista_muestras.append(volt)
            print ('voltage = ' + str(volt))
           
        tabla_muestras.append(lista_muestras) 
        duracion_barrido= time.time() - start_time
    with open("pruebav1" + str(barrido) + "-" +str(duracion_barrido) + ".csv", "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerows(tabla_muestras)

barrido 1
freq 1
voltage = 9.12605572
freq 2
voltage = 9.12602997


In [45]:
import os

In [46]:
os.getcwd()

'C:\\Users\\khemi\\Lock-in Control\\SR830_LockInAmplifier'